Introduction:

Metro plays an important role in modern transportation, efficienly moving millions of commuters daily. Ensuring the safety and reliability of these metro system is utmost important. An unexpected anomalies and failure can cause inconvenience to the passengers, moreover it can pose significant economic and safety risk.

The MetroPT3 dataset at our disposal contains columns representing various operational features such as TP2, TP3, H1, DV_Pressure, Reservoirs, Oil_Temperature, Motor_Current, COMP, DV_electric, Towers, MPG, LPS, Pressure, Oil_level, and cadual_impulses. These columns represents several data points that are continuously collected by different sensors and monitoring systems deployed in the metro.


Objective:

The primary objective of this research project is to harness the potential of machine learning algorithms developed to detect the anomalies to identify the deviations from normal patterns within the metro system.
 
Anomalies includes sudden spike or drop in pressure, irregular temperature patterns, unexpected motor current fluctuations etc. Detecting such anomalies is very crucial to ensure reliability and safety of the metro system.

Terminologies:

Analog Sensors : As for the analog sensors, we have pressure, temperature and electric current consumed at different components of the APU, as detailed below.

1. TP2 : Pressure on compressor (bar).

2. TP3 : Pressure generated at pneumatic panel (bar).

3. H1 : Valve that is activated when the pressure read by the pressure switch of the command is above the      operating pressure of 10.2 bar.

4. DV_Pressure : Pressure exerted due to pressure drop generated when air dryers towers discharge the water. When it is equal to zero, the compressor is working under load (bar).

5. Reservoirs : Pressure inside the air tanks installed on the trains (bar).

6. Oil_temperature : Temperature of oil present on the compressor (°C).

7. Motor_current : Motor’s current, which should present the following values: 
(i) close to 0 A when the compressor turns off; 
(ii) close to 4 A when the compressor is working offloaded; and 
(iii) close to 7 A when the compressor is operating under load (A);

Digital Sensors : The eight digital signals are collected directly from the APU and GPS information.

The digital sensors installed in the APU assume only two different values: zero when inactive or one when a specific event activates them. The considered digital sensors are the following.

8. COMP : Electrical signal of the air intake valve on the compressor. It is active when there is no admission of air on the compressor, meaning that the compressor turns off or working offloaded.

9. DV_electric : Electrical signal that commands the compressor outlet valve. When it is active, it means that the compressor is working under load; when it is not active, it means that the compressor is off or offloaded.s

10. TOWERS : Signal that defines which tower is drying the air and which tower is draining the humidity removed from the air. When it is not active, it means that tower one is working; when it is active, it means that tower two is working.

11. MPG : Is responsible for activating the intake valve to start the compressor under load when the pressure in the APU is below 8.2 bar. Consequently, it will activate the sensor COMP, which assumes the same behaviour as the MPG sensor.

12. LPS : Signal activated when the pressure is lower than 7 bars.

13. Pressure_switch : Signal activated when pressure is detected on the pilot control valve.

14. Oil_Level : The oil level on the compressor is active (equal to one) when the oil is below the expected values.

15. Caudal_impulses : Signal produced by the flowmeter indicating the existence of the flow of air per second.